In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter, freqz, filtfilt
import math 
import pickle
from pprint import pp
from datetime import datetime

from scipy.stats import linregress, kurtosis, skew
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

%matplotlib inline
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; } .CodeMirror pre {font-size: 9pt;}</style>"))

In [4]:
devices = ['L', 'R', 'F'] # left, right, front
wavelengths = ['740', '850', '940'] # nm
optodes = ['10', '27'] # mm separation from emmitter

In [5]:
pd.options.mode.chained_assignment = None
  
def filter(data, lowcut, highcut):
  fs = 25 # 25 Hz  (40ms intervals)
  order = 6

  nyq = 0.5 * fs
  low = lowcut / nyq
  high = highcut / nyq
  b, a = butter(order, [low, high], btype='band') 
  y = filtfilt(b, a, data)
  return y

def filter_df(df, lowcut, highcut):
  for dev in devices: # L, R, or F
    for opt in optodes: # 10 or 27 mm
      for wl in wavelengths: # 740, 850, or 940 nm 
        df[f'{wl}nm{opt}mm_{dev}_filt'] = filter(df[f'{wl}nm{opt}mm_{dev}'], lowcut, highcut) 


In [7]:
def merge_dfs(subject):
  dfs = []
  for dev in devices:
    dev_df = pd.read_csv(f'./data/{subject}/blueberry/0{dev}.csv')
    dev_df = dev_df[dev_df.timestamp != 'timestamp']   # remove headers (inserted when device reconnects?)
    dev_df = dev_df.apply(pd.to_numeric)   # objects -> float64 & int64

    dev_df['timestamp'] = dev_df.timestamp.round(3)
    dev_df['datetime'] = dev_df.timestamp.apply(datetime.fromtimestamp)
    dev_df.set_index('datetime', inplace=True)

    dev_df = dev_df[~dev_df.index.duplicated()]
    dev_df = dev_df.add_suffix(f'_{dev}')
    dfs.append(dev_df)

  df = pd.concat(dfs, axis=1)
  df.interpolate(method='index', axis=0, inplace=True)
  df.reset_index(inplace=True)
  df.dropna(inplace=True)

  df['timestamp'] = df.datetime.apply(datetime.timestamp)
  df.drop(df[df.index%3 != 0].index, inplace=True)
  df.reset_index(drop=True, inplace=True)

  return df

In [ ]:
def segment_df(df, det, data_dict):
    # pp(opt)
    # pp(df.columns)
    stimuli = ['visual', 'auditory', 'mental']
    for stim in stimuli:
      # create df for each period in parts 1-3 (regular timing) and insert into dict
      data_dict[f'{stim}_regular_math'] = []
      data_dict[f'{stim}_regular_rest'] = []
      for offset in range(0, det['regular_part_time'], 2*det['period_time']):
        math_start = det[f'{stim}_regular_start'] + offset - 5
        math_end = det[f'{stim}_regular_start'] + offset + det['period_time'] + 5
        math_df = df[ (df['timestamp'] > math_start) & (df['timestamp'] < math_end) ]
        data_dict[f'{stim}_regular_math'].append(math_df)
        
        rest_start = det[f'{stim}_regular_start'] + offset + det['period_time'] - 5
        rest_end = det[f'{stim}_regular_start'] + offset + 2*det['period_time'] + 5
        rest_df = df[ (df['timestamp'] > rest_start) & (df['timestamp'] < rest_end) ]
        data_dict[f'{stim}_regular_rest'].append(rest_df)

      # create df for each period in parts 4-6 (random timing) and insert into dict
      data_dict[f'{stim}_random_math'] = []
      data_dict[f'{stim}_random_rest'] = []
      for math_offset, rest_offset in zip(det[f'{stim}_random_math_starts'], det[f'{stim}_random_rest_starts']):
        math_start = det[f'{stim}_random_start'] + math_offset - 5
        math_end = det[f'{stim}_random_start'] + math_offset + det['period_time'] + 5
        math_df = df[ (df['timestamp'] > math_start) & (df['timestamp'] < math_end) ]
        data_dict[f'{stim}_random_math'].append(math_df)

        rest_start = det[f'{stim}_random_start'] + rest_offset - 5
        rest_end = det[f'{stim}_random_start'] + rest_offset + det['period_time'] + 5
        rest_df = df[ (df['timestamp'] > rest_start) & (df['timestamp'] < rest_end) ]
        data_dict[f'{stim}_random_rest'].append(rest_df)

       
    # pp(data_dict)



In [ ]:
def calc_dpf(age, wl): 
    # equation from [DOI: 10.1117/1.JBO.18.10.105004]
    dpf = 223.3 + (0.05624 * age**0.8493) - (5.723 * 10**-7 * int(wl)**3) + (0.001245 * int(wl)**2) - (0.9025*int(wl))
    return dpf

In [ ]:
# calculate concentration of oxygenated (HbO) and deoxygenated (HbR) hemoglobin
def calculate_hb(df, age):
    # TODO
    # either do all at once like below or sequentially by calculating relative change at each timepoint like swift code?

    # separtion between emitter and dectectors in mm
    l1 = 10.3 
    l2 = 27.0 

    # extinction coefficients (https://omlc.org/spectra/hemoglobin/summary.html)
    e_hbo_740 = 0.446
    e_hbr_740 = 1.11588

    e_hbo_850 = 1.058
    e_hbr_850 = 0.69132

    e_hbo_940 = 1.214
    e_hbr_940 = 0.69344

    E = np.array([[e_hbo_740, e_hbr_740],
                  [e_hbo_850, e_hbr_850],
                  [e_hbo_940, e_hbr_940]])


    for dev in devices:
        for opt in optodes:
            B = []
            dODs = []
            dpfs  = []
            for wl in wavelengths:
                I = df[f'{wl}nm{opt}mm_{dev}'].values / df[f'{wl}nm{opt}mm_{dev}'].mean() # normalized light intensity 
                dODs.append( -np.log10(I) ) # change in optical density  LN?
                dpfs.append( calc_dpf(int(wl), age) )

            print(dODs)
            print(dpfs)

            #     B.append(dOD/dpf)
            # B = np.array(B)
            # print(B)

            dC = 1/int(opt) * (E * dpfs) * np.dot(np.linalg.inv(A), B) # change in concentrations
            # df[f'hbo{opt}mm_{dev}'] = dC[0]
            # df[f'hbr{opt}mm_{dev}'] = dC[1]

In [ ]:
def main():
  data_dict = {}
  subjects = ['khiem']#,'zoey','daniel', 'david', 'nader']
  for subject in subjects:
    with open(f'./data/{subject}/info.json', 'r') as file:
      info = json.load(file)

    df = create_df(subject)
    display(df)

    auditory_random_df = df[ (df.timestamp > info['auditory_random_start'] - 8000) & (df.timestamp < info['auditory_random_start']) ]
    
    filter_df(df, 0.01, 0.5) # not sure about these values 
    estimate_hb(df, 21)

    print(df.columns)
    # plot_hb(df)

main()

In [ ]:
with open(''